<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Agentic-RAG-Streamlit-App/blob/main/agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# agentic_rag.ipynb

# 1️⃣ Установка и импорт
!pip install -q langchain langchain-groq tavily-python faiss-cpu python-dotenv

import os
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.embeddings import GroqEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import Groq

# 2️⃣ Загрузка ключей из .env
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")  # опционально
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")  # опционально

assert GROQ_API_KEY, "GROQ_API_KEY не задан"
assert TAVILY_API_KEY, "TAVILY_API_KEY не задан"

# 3️⃣ Инициализация LLM и эмбеддингов
llm = Groq(model="gpt-3.5", api_key=GROQ_API_KEY)

embeddings = GroqEmbeddings(api_key=GROQ_API_KEY)

# 4️⃣ Загружаем/создаём KB
# Пример: список документов
kb_docs = [
    {"text": "Agentic AI systems follow the loop: interpret goal, plan, act, reflect, answer.", "source": "kb:agentic"},
    {"text": "Tools should be used only when necessary.", "source": "kb:tools"},
    {"text": "Always cite sources like KB or web pages.", "source": "kb:citation"}
]

# Создание векторного индекса
texts = [d["text"] for d in kb_docs]
vectorstore = FAISS.from_texts(texts, embeddings)

# 5️⃣ Создание RetrievalQA chain
prompt_template = """
Answer the user question based on the retrieved documents.
Cite sources inline using [source] notation.
Keep answer concise (2–4 sentences).

Question: {question}
Answer:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["question"])

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 6️⃣ Функция для Streamlit
def answer_question(query: str):
    """
    Возвращает ответ агента на запрос query.
    Результат содержит текст и источники.
    """
    res = qa_chain(query)
    answer_text = res['result']
    sources = [doc.metadata.get('source', f"doc{i}") for i, doc in enumerate(res['source_documents'])]

    # Форматирование
    answer_with_sources = answer_text + "\n\nSources:\n" + "\n".join(f"[{i+1}] {s}" for i, s in enumerate(sources))
    return answer_with_sources

# 7️⃣ Тест
questions = [
    "What is an agentic AI loop?",
    "When should an agent use tools?",
    "How should sources be cited?"
]

for q in questions:
    print("Q:", q)
    print(answer_question(q))
    print("---")
